In [1]:
%matplotlib inline
import missingno as mso

ModuleNotFoundError: No module named 'missingno'

In [2]:
import pandas as pd
from glob import glob
from pypcleaner import convert_excel_time

In [3]:
df = pd.read_hdf('big_multi_index.h5')
scans = pd.read_pickle('scans.p')
imaging_id_codes = pd.read_pickle('imaging_id_codes.p')
id_codes = imaging_id_codes.loc[scans.index,'prefix']
scan_id_df = df.loc[scans.index]
scan_id_df.drop('day_0', axis='columns', level=0, inplace = True)

drop_these = ['bp_three_hr','bp_three_hr_systolic','bp_three_hr_diastolic','bp_three_hr_datetime',
              'bp_seven_hr','bp_seven_hr_systolic','bp_seven_hr_diastolic','bp_seven_hr_datetime',
              'bp_other_hr','bp_other_hr_systolic','bp_other_hr_diastolic','bp_other_hr_datetime',
              'hrt_seven_hr','hrt_seven_hr_rate','hrt_seven_hr_datetime','hrt_other_hr',
              'hrt_other_hr_rate','hrt_other_hr_datetime','surgical_intervention',
              'surgical_intervention_detail','invasive_intervention','invasive_intervention_detail',
              'imaging_intervention','imaging_intervention_detail',
              'other_intervention','other_intervention_detail','medication_given','medication_comment']

scan_id_df.drop([('day_baseline','recent_medical_history',c) for c in drop_these], axis = 'columns',inplace = True)

scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')] = \
scan_id_df[('day_baseline','recent_medical_history','tpa_datetime')].apply(convert_excel_time).dt.round('5min')

In [4]:
remove_tests = \
[('day_baseline','physical_examination'), 
 ('day_baseline', 'past_medical_history'), 
 ('day_baseline', 'CCMI'), 
 ('day_baseline', 'further_stroke_event_part'), 
 ('day_baseline', 'bloods'), 
 ('day_baseline', 'laboratory_tests'), 
 ('day_baseline', 'radiological_scans'), 
 ('day_baseline', 'concomitant_medications'), 
 ('day_baseline', 'pre-stroke_mRS'),
 ('day_7', 'bloods') , 
 ('day_7', 'history_of_depression'), 
 ('day_7', 'physical_risk_factors'), 
 ('day_7', 'diet_q'),
 ('day_7', 'laboratory_tests'), 
 ('day_7', 'concomitant_medications'), 
 ('day_7', 'concomitant_medications_plus'), 
 ('day_90', 'bloods'), 
 ('day_90', 'physical_examination'), 
 ('day_90', 'physical_risk_factors'), 
 ('day_90', 'diet_q'), 
 ('day_90', 'laboratory_tests'), 
 ('day_90', 'further_stroke_event_part'), 
 ('day_90', 'history_of_depression'), 
 ('day_90', 'concomitant_medications'), 
 ('day_365', 'bloods'), 
 ('day_365', 'physical_examination'), 
 ('day_365', 'physical_risk_factors'), 
 ('day_365', 'diet_q'), 
 ('day_365', 'laboratory_tests'), 
 ('day_365', 'further_stroke_event_part'), 
 ('day_365', 'history_of_depression'), 
 ('day_365', 'concomitant_medications'), 
 ('day_1', 'vital_signs'), 
 ('day_1', 'bloods'), 
 ('day_1', 'laboratory_tests'), 
 ('day_1', 'serious_adverse_event')]

scan_id_df.drop(remove_tests, axis = 'columns',inplace = True)

/Users/alistairwalsh/anaconda3/envs/START/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: PerformanceWarning: indexing past lexsort depth may impact performance.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
demographics = pd.read_pickle('base_df_data.p')


stroke_onset = demographics.loc[scans.index,['identity_stroke_onset']]
multi_NIHSS = scan_id_df[('day_baseline','NIHSS_multiple')]
multi_with_stroke_onset = stroke_onset.join(multi_NIHSS)

for c in ['nihss_three_hr_datetime', 'nihss_seven_hr_datetime', 
          'nihss_eighteen_hr_datetime', 'nihss_other_hr_datetime']:
    hrs = multi_with_stroke_onset[c] - multi_with_stroke_onset['identity_stroke_onset']
    multi_with_stroke_onset[c] = hrs.dt.round('5min')
    
scan_id_df[('day_baseline','NIHSS_multiple')] = multi_with_stroke_onset

/Users/alistairwalsh/anaconda3/envs/START/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)
/Users/alistairwalsh/anaconda3/envs/START/lib/python3.7/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
/Users/alistairwalsh/anaconda3/envs/START/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2880: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [6]:
bit_1_cols = demographics.loc[:,'identity_gender':'identity_age_at_stroke_in_years'].columns
bit_2_cols = demographics.loc[:,'demographics_ancestry_1':].columns
useful_columns = bit_1_cols.append(bit_2_cols)
demographics = demographics.loc[scans.index,useful_columns]

In [7]:
demographics['identity_gender'] = demographics['identity_gender'].map({1:'female',0:'male'})

In [8]:
id_codes = pd.read_pickle('imaging_id_codes.p').loc[scans.index,['prefix']].join(scan_id_df[('day_1','infarct_type')])
#infarct_type = scan_id_df[('day_1','infarct_type')]

/Users/alistairwalsh/anaconda3/envs/START/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2854: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


In [9]:
id_codes.columns = pd.MultiIndex.from_product([['day_90'],['lesion_location_confirmed'], id_codes.columns])

In [10]:
new_bit = scan_id_df['day_90']['lesion_location_confirmed'].copy()

In [11]:
new_bit.columns = pd.MultiIndex.from_product([['day_90'],['lesion_location_confirmed'],
                            new_bit.columns])

In [12]:
#scan_id_df['day_90']['lesion_location_confirmed'] = \
new_bit = new_bit.join(id_codes)

In [13]:
scan_id_df.drop(('day_90',
  'lesion_location_confirmed',
  'lesion location confirmed by imaging at 3 months (BC report)'), axis = 'columns',inplace = True)

In [14]:
scan_id_df = scan_id_df.join(new_bit)

In [15]:
scan_id_df[('day_90','TUGT_average','correct_average')] = \
scan_id_df['day_90']['TUGT_average'][['time1', 'time2', 'time3']].mean(axis = 'columns').round(1)
scan_id_df[('day_365','TUGT','correct_average')] = \
scan_id_df['day_365']['TUGT'][['time1', 'time2', 'time3']].mean(axis='columns').round(1)

bad = \
[('day_90', 'TUGT', 'time_taken'),
 ('day_90', 'TUGT_average', 'time1'),
 ('day_90', 'TUGT_average', 'time2'),
 ('day_90', 'TUGT_average', 'time3'),
 ('day_90', 'TUGT_average', 'TUGT_average'),
 ('day_365', 'TUGT', 'time1'),
 ('day_365', 'TUGT', 'time2'),
 ('day_365', 'TUGT', 'time3'),
 ('day_365', 'TUGT', 'average')]
scan_id_df.drop(bad,axis='columns',inplace=True)

In [16]:
# drop mri
mri_drop = scan_id_df.filter(regex='MRI').columns.tolist()
scan_id_df.drop(mri_drop,axis='columns',inplace=True)

In [17]:
dont_delete = [('day_7', 'RAPA', 'resp_date'),('day_90', 'RAPA', 'resp_date'),
('day_90', 'WSAS', 'resp_date.2'),('day_365', 'RAPA', 'resp_date'),
 ('day_365', 'WSAS', 'resp_date.2')]
response_drop = \
[('day_7', 'RAPA', 'resp'),
 ('day_7', 'RAPA', 'resp_other'),
 ('day_90', 'mRS', 'resp'),
 ('day_90', 'mRS', 'resp_other'),
 ('day_90', 'BI', 'resp.1'),
 ('day_90', 'BI', 'resp_other.1'),
 ('day_90', 'SIS', 'resp.2'),
 ('day_90', 'SIS', 'resp_other.2'),
 ('day_90', 'RAPA', 'resp'),
 ('day_90', 'RAPA', 'resp_other'),
 ('day_90', 'WSAS', 'resp.5'),
 ('day_90', 'WSAS', 'resp_other.5'),
 ('day_365', 'mRS', 'resp'),
 ('day_365', 'mRS', 'resp_other'),
 ('day_365', 'BI', 'resp.1'),
 ('day_365', 'BI', 'resp_other.1'),
 ('day_365', 'SIS', 'resp.2'),
 ('day_365', 'SIS', 'resp_other.2'),
 ('day_365', 'RAPA', 'resp'),
 ('day_365', 'RAPA', 'resp_other'),
 ('day_365', 'WSAS', 'resp.5'),
 ('day_365', 'WSAS', 'resp_other.5')]

scan_id_df.drop(response_drop,axis='columns',inplace=True)
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

In [18]:
drop_nihss_multiple = [('day_baseline','NIHSS_multiple',c) for c in ['nihss_three_hr',
                                                                     'nihss_seven_hr',
                                                                     'nihss_eighteen_hr',
                                                                     'nihss_other_hr']]

scan_id_df.drop(drop_nihss_multiple,axis = 'columns',inplace= True)

In [19]:
# RAVENs had no data
orig_df = pd.read_excel('data/extract/START 12 months (20161121).xlsx')
ravens = orig_df.iloc[scans.index,689:762]
ravens.rename(columns = {'RAVENS_total_score_12mth':'score'}, inplace = True)
scan_id_df[('day_365','RAVENs')] = ravens

/Users/alistairwalsh/anaconda3/envs/START/lib/python3.7/site-packages/IPython/core/async_helpers.py:68: PerformanceWarning: indexing past lexsort depth may impact performance.
  coro.send(None)
/Users/alistairwalsh/anaconda3/envs/START/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2880: PerformanceWarning: indexing past lexsort depth may impact performance.
  return runner(coro)


In [20]:
scan_id_df.columns = scan_id_df.columns.remove_unused_levels()

In [21]:
scan_id_df.shape

(68, 1679)

In [22]:
scan_id_df.columns.tolist()

[('day_baseline', 'NIHSS_multiple', 'nihss_three_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_three_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_seven_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_eighteen_hr_datetime'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr_score'),
 ('day_baseline', 'NIHSS_multiple', 'nihss_other_hr_datetime'),
 ('day_baseline', 'recent_medical_history', 'tpa'),
 ('day_baseline', 'recent_medical_history', 'tpa_datetime'),
 ('day_baseline', 'recent_medical_history', 'tpa_volume'),
 ('day_baseline', 'NIHSS', 'loc'),
 ('day_baseline', 'NIHSS', 'loc_quest'),
 ('day_baseline', 'NIHSS', 'loc_command'),
 ('day_baseline', 'NIHSS', 'best_gaze'),
 ('day_baseline', 'NIHSS', 'vis_field_test'),
 ('day_baseline', 'NIHSS', 'facial_palsy'),
 ('day_baseline', 'NIHSS', 'motor_arm_l'),
 ('day_baseline', 'NIHSS', 'm

In [23]:
scans

,p_participant_num,site_id,initials,prefix,path
119,01,01,JW,D_PP_P01-01-JW,[build_dataset/correlation_maps/D_PP_P01-01-JW...
123,01,02,AS,D_PP_P02-01-AS,[build_dataset/correlation_maps/D_PP_P02-01-AS...
124,01,12,JW,D_PP_P12-01-JW,[build_dataset/correlation_maps/D_PP_P12-01-JW...
126,10,01,NP,D_PP_P01-10-NP,[build_dataset/correlation_maps/D_PP_P01-10-NP...
127,02,12,AS,D_PP_P12-02-AS,[build_dataset/correlation_maps/D_PP_P12-02-AS...
129,07,01,GS,D_PP_P01-07-GS,[build_dataset/correlation_maps/D_PP_P01-07-GS...
132,02,02,SH,D_PP_P02-02-SH,[build_dataset/correlation_maps/D_PP_P02-02-SH...
133,11,01,GW,D_PP_P01-11-GW,[build_dataset/correlation_maps/D_PP_P01-11-GW...
136,01,15,EM,D_PP_P15-01-EM,[build_dataset/correlation_maps/D_PP_P15-01-EM...
137,02,15,DK,D_PP_P15-02-DK,[build_dataset/correlation_maps/D_PP_P15-02-DK...


In [28]:
scan_id_df.columns.levels[0]

Index(['day_1', 'day_365', 'day_7', 'day_90', 'day_baseline'], dtype='object')

In [29]:
scan_id_df.columns.levels[1]

Index(['NIHSS_multiple', 'recent_medical_history', 'NIHSS', 'MoCA', 'MADRS',
       'RAPA', 'infarct_type', 'mRS', 'BI', 'SIS', 'WSAS', 'MMSE',
       'digit_span', 'RAVENs', 'STROOP', 'shape_cancellation', 'TDT', 'ARAT',
       'ACS', 'trail_making_test_b', 'lesion_location_confirmed',
       'TUGT_average', 'TUGT'],
      dtype='object')

In [30]:
scan_id_df.columns.levels[2]

Index(['nihss_three_hr_score', 'nihss_three_hr_datetime',
       'nihss_seven_hr_score', 'nihss_seven_hr_datetime',
       'nihss_eighteen_hr_score', 'nihss_eighteen_hr_datetime',
       'nihss_other_hr_score', 'nihss_other_hr_datetime', 'tpa',
       'tpa_datetime',
       ...
       'given up_childcare', 'beginning to do_childcare', 'new_childcare',
       'childcare_caln', 'childcare_pal', 'childcare_raln', 'date_time',
       'lesion location confirmed by imaging at 3 months (BC report)',
       'prefix', 'correct_average'],
      dtype='object', length=788)

In [54]:
scan_id_df.loc[:,['day_365','day_90','day_7']]

day_7                                                              \
    NIHSS                                                               
      loc loc_quest loc_command best_gaze vis_field_test facial_palsy   
119   0.0       0.0         0.0       0.0            0.0          0.0   
123   0.0       0.0         0.0       0.0            0.0          1.0   
124   0.0       0.0         0.0       0.0            0.0          1.0   
126   0.0       0.0         0.0       0.0            0.0          2.0   
127   0.0       0.0         0.0       0.0            0.0          2.0   
129   0.0       0.0         0.0       0.0            0.0          0.0   
132   0.0       0.0         0.0       0.0            0.0          0.0   
133   0.0       0.0         0.0       0.0            0.0          0.0   
136   0.0       0.0         0.0       0.0            0.0          1.0   
137   0.0       0.0         0.0       0.0            0.0          2.0   
138   0.0       0.0         0.0       0.0            0.0          1.0   
139   0.0       0.0         0.0       0.0            0.0          1.0   
140   0.0       0.0         0.0       0.0            0.0          0.0   
141   0.0       0.0         0.0       0.0            0.0          0.0   
142   0.0       0.0         0.0       0.0            0.0          0.0   
143   0.0       0.0         0.0       0.0            0.0          0.0   
144   0.0       0.0         0.0       0.0            0.0          2.0   
146   0.0       0.0         0.0       0.0            0.0          1.0   
147   0.0       0.0         0.0       0.0            0.0          0.0   
148   0.0       0.0         0.0       0.0            0.0          0.0   
151   0.0       0.0         0.0       0.0            0.0          0.0   
152   0.0       0.0         0.0       1.0            0.0          1.0   
154   0.0       0.0         0.0       0.0            0.0          1.0   
155   0.0       0.0         0.0       0.0            0.0          0.0   
156   0.0       0.0         0.0       0.0            0.0          0.0   
157   0.0       0.0         0.0       1.0            0.0          0.0   
159   0.0       0.0         0.0       0.0            0.0          0.0   
161   0.0       0.0         0.0       0.0            0.0          0.0   
162   0.0       0.0         0.0       0.0            0.0          1.0   
163   0.0       0.0         0.0       0.0            2.0          0.0   
..    ...       ...         ...       ...            ...          ...   
178   0.0       0.0         0.0       0.0            0.0          0.0   
180   0.0       0.0         0.0       0.0            0.0          0.0   
182   0.0       0.0         0.0       0.0            0.0          0.0   
183   0.0       0.0         0.0       0.0            0.0          0.0   
186   0.0       0.0         0.0       0.0            0.0          1.0   
188   0.0       0.0         0.0       0.0            0.0          0.0   
189   0.0       0.0         0.0       0.0            0.0          0.0   
190   0.0       0.0         0.0       0.0            0.0          0.0   
191   0.0       0.0         0.0       0.0            0.0          0.0   
192   0.0       0.0         0.0       0.0            0.0          0.0   
194   NaN       NaN         NaN       NaN            NaN          NaN   
196   0.0       0.0         0.0       0.0            0.0          0.0   
197   0.0       0.0         0.0       0.0            0.0          0.0   
198   0.0       0.0         0.0       0.0            0.0          0.0   
199   0.0       0.0         0.0       0.0            0.0          0.0   
200   0.0       0.0         0.0       0.0            0.0          0.0   
201   0.0       0.0         0.0       0.0            0.0          1.0   
202   0.0       0.0         0.0       0.0            0.0          0.0   
204   0.0       0.0         0.0       0.0            0.0          0.0   
205   0.0       0.0         0.0       0.0            0.0          0.0   
206   0.0       0.0         0.0       0.0            0.0      

In [56]:
from IPython.display import display

In [76]:
for day in ['day_baseline', 'day_1', 'day_7', 'day_90', 'day_365']:
    df = scan_id_df.loc[:,day]
    print(day)
    headings = set(df.columns.get_level_values(0))
    for h in headings:
        print(h,'\n',df[h].columns.get_level_values(0).values)
    display(df.head())

day_baseline
NIHSS 
 ['loc' 'loc_quest' 'loc_command' 'best_gaze' 'vis_field_test'
 'facial_palsy' 'motor_arm_l' 'motor_arm_r' 'motor_leg_l' 'motor_leg_r'
 'limb_ataxia' 'sensory' 'best_lang' 'dysarthria' 'extinction' 'score'
 'valid' 'reason']
recent_medical_history 
 ['tpa' 'tpa_datetime' 'tpa_volume']
NIHSS_multiple 
 ['nihss_three_hr_score' 'nihss_three_hr_datetime' 'nihss_seven_hr_score'
 'nihss_seven_hr_datetime' 'nihss_eighteen_hr_score'
 'nihss_eighteen_hr_datetime' 'nihss_other_hr_score'
 'nihss_other_hr_datetime']


NIHSS_multiple                                               \
    nihss_three_hr_score nihss_three_hr_datetime nihss_seven_hr_score   
119                  NaN                     NaT                  NaN   
123                  NaN                     NaT                  NaN   
124                  NaN                     NaT                  NaN   
126                  NaN                     NaT                  NaN   
127                  NaN                     NaT                  NaN   

                                                     \
    nihss_seven_hr_datetime nihss_eighteen_hr_score   
119                     NaT                     NaN   
123                     NaT                     NaN   
124                     NaT                     NaN   
126                     NaT                     NaN   
127                     NaT                     NaN   

                                                                             \
    nihss_eighteen_hr_datetime nihss_other_hr_score nihss_other_hr_datetime   
119                        NaT                  NaN                     NaT   
123                        NaT                  NaN                     NaT   
124                        NaT                  NaN                     NaT   
126                        NaT                  NaN                     NaT   
127                        NaT                  NaN                     NaT   

    recent_medical_history               ...       NIHSS              \
                       tpa tpa_datetime  ... motor_leg_l motor_leg_r   
119                    NaN          NaT  ...         0.0         1.0   
123                    NaN          NaT  ...         3.0         0.0   
124                    NaN          NaT  ...         0.0         0.0   
126                    NaN          NaT  ...         0.0         0.0   
127                    NaN          NaT  ...         4.0         0.0   

                                                                            
    limb_ataxia sensory best_lang dysarthria extinction score valid reason  
119         2.0     0.0       0.0        1.0        0.0   5.0   1.0    NaN  
123         0.0     2.0       0.0        1.0        0.0  11.0   1.0    NaN  
124         0.0     0.0       2.0        1.0        0.0   8.0   1.0    NaN  
126         0.0     0.0       0.0        2.0        0.0   4.0   1.0    NaN  
127         0.0     1.0       0.0        2.0        1.0  22.0   1.0    NaN  

[5 rows x 29 columns]

day_1
infarct_type 
 ['is_ich' 'date_time' 'symptomatic' 'loc_frontal' 'loc_temporal'
 'loc_parietal' 'loc_occipital' 'loc_subcortical' 'loc_brainstem'
 'loc_cerebellar' 'loc_subdural' 'loc_subarachnoid' 'treatment'
 'treatment_spec']
NIHSS 
 ['loc' 'loc_quest' 'loc_command' 'best_gaze' 'vis_field_test'
 'facial_palsy' 'motor_arm_l' 'motor_arm_r' 'motor_leg_l' 'motor_leg_r'
 'limb_ataxia' 'sensory' 'best_lang' 'dysarthria' 'extinction' 'score'
 'valid' 'reason']


NIHSS                                                              \
      loc loc_quest loc_command best_gaze vis_field_test facial_palsy   
119   0.0       0.0         0.0       0.0            0.0          0.0   
123   0.0       0.0         0.0       0.0            0.0          1.0   
124   0.0       2.0         0.0       0.0            0.0          1.0   
126   0.0       0.0         0.0       0.0            0.0          2.0   
127   1.0       0.0         0.0       1.0            2.0          2.0   

                                                     ... infarct_type  \
    motor_arm_l motor_arm_r motor_leg_l motor_leg_r  ... loc_temporal   
119         0.0         1.0         0.0         1.0  ...          0.0   
123         3.0         0.0         2.0         0.0  ...          0.0   
124         0.0         1.0         0.0         1.0  ...          0.0   
126         0.0         0.0         0.0         0.0  ...          0.0   
127         1.0         0.0         1.0         0.0  ...          0.0   

                                                                             \
    loc_parietal loc_occipital loc_subcortical loc_brainstem loc_cerebellar   
119          0.0           0.0             0.0           0.0            0.0   
123          0.0           0.0             0.0           0.0            0.0   
124          0.0           0.0             0.0           0.0            0.0   
126          0.0           0.0             0.0           0.0            0.0   
127          0.0           0.0             0.0           0.0            0.0   

                                                            
    loc_subdural loc_subarachnoid treatment treatment_spec  
119          0.0              0.0       NaN            NaN  
123          0.0              0.0       NaN            NaN  
124          0.0              0.0       NaN            NaN  
126          0.0              0.0       NaN            NaN  
127          0.0              0.0       NaN            NaN  

[5 rows x 32 columns]

day_7
NIHSS 
 ['loc' 'loc_quest' 'loc_command' 'best_gaze' 'vis_field_test'
 'facial_palsy' 'motor_arm_l' 'motor_arm_r' 'motor_leg_l' 'motor_leg_r'
 'limb_ataxia' 'sensory' 'best_lang' 'dysarthria' 'extinction' 'score'
 'valid' 'reason']
MADRS 
 ['report_sad' 'apparent_sad' 'inner_tension' 'reduce_sleep'
 'reduce_appetite' 'concent_diff' 'lassitude' 'inable_feel' 'pess_thought'
 'suicide_thought' 'score' 'valid' 'reason']
infarct_type 
 ['is_ich' 'date_time.1' 'symptomatic' 'loc_frontal' 'loc_temporal'
 'loc_parietal' 'loc_occipital' 'loc_subcortical' 'loc_brainstem'
 'loc_cerebellar' 'loc_subdural' 'loc_subarachnoid' 'treatment'
 'treatment_spec']
MoCA 
 ['exe_trail' 'exe_cube' 'exe_clock_cont' 'exe_clock_num' 'exe_clock_hand'
 'name_lion' 'name_rhino' 'name_camel' 'mem_face_1' 'mem_velvet_1'
 'mem_church_1' 'mem_daisy_1' 'mem_red_1' 'mem_face_2' 'mem_velvet_2'
 'mem_church_2' 'mem_daisy_2' 'mem_red_2' 'att_forward' 'att_backward'
 'att_a' 'att_subtract' 'lang1' 'lang2' 'lang3' 'abs_t

NIHSS                                                              \
      loc loc_quest loc_command best_gaze vis_field_test facial_palsy   
119   0.0       0.0         0.0       0.0            0.0          0.0   
123   0.0       0.0         0.0       0.0            0.0          1.0   
124   0.0       0.0         0.0       0.0            0.0          1.0   
126   0.0       0.0         0.0       0.0            0.0          2.0   
127   0.0       0.0         0.0       0.0            0.0          2.0   

                                                     ... infarct_type  \
    motor_arm_l motor_arm_r motor_leg_l motor_leg_r  ... loc_temporal   
119         0.0         1.0         0.0         0.0  ...          0.0   
123         3.0         0.0         2.0         0.0  ...          0.0   
124         0.0         0.0         0.0         0.0  ...          0.0   
126         0.0         0.0         0.0         0.0  ...          0.0   
127         1.0         0.0         1.0         0.0  ...          0.0   

                                                                             \
    loc_parietal loc_occipital loc_subcortical loc_brainstem loc_cerebellar   
119          0.0           0.0             0.0           0.0            0.0   
123          0.0           0.0             0.0           0.0            0.0   
124          0.0           0.0             0.0           0.0            0.0   
126          0.0           0.0             0.0           0.0            0.0   
127          0.0           0.0             0.0           0.0            0.0   

                                                            
    loc_subdural loc_subarachnoid treatment treatment_spec  
119          0.0              0.0       NaN            NaN  
123          0.0              0.0       NaN            NaN  
124          0.0              0.0       NaN            NaN  
126          0.0              0.0       NaN            NaN  
127          0.0              0.0       NaN            NaN  

[5 rows x 97 columns]

day_90
MMSE 
 ['age_assessment' 'pencil' 'watch' 'school_years' 'time_year'
 'time_season' 'time_month' 'time_weekday' 'time_date' 'place_state'
 'place_country' 'place_city' 'place_building' 'place_floor'
 'register_apple' 'register_penny' 'register_table' 'serial7_93'
 'serial7_86' 'serial7_79' 'serial7_72' 'serial7_65' 'world_d' 'world_l'
 'world_r' 'world_o' 'world_w' 'recall_apple' 'recall_penny'
 'recall_table' 'naming_pencil' 'naming_watch' 'repetition'
 'comprehension_take_paper' 'comprehension_fold_paper'
 'comprehension_put_paper' 'reading' 'writing' 'drawing' 'score'
 'consciousness']
SIS 
 ['phy_arm' 'phy_hand' 'phy_leg' 'phy_foot' 'mem_told' 'mem_day_bef'
 'mem_to_do' 'mem_wk_day' 'mem_concert' 'mem_think_quick' 'mem_sol_prob'
 'feel_sad' 'feel_nobody' 'feel_burden' 'feel_nothing' 'feel_blame'
 'feel_enjoy' 'feel_nervous' 'feel_life_with' 'feel_smile' 'com_say_name'
 'com_under' 'com_reply' 'com_correct' 'com_part' 'com_ph' 'com_ph_num'
 'act_knife' 'act_dress' 'act_bathe'

mRS                   NIHSS                                  \
    score date valid reason   loc loc_quest loc_command best_gaze   
119   1.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
123   4.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
124   1.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
126   2.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
127   2.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   

                                 ... lesion_location_confirmed                \
    vis_field_test facial_palsy  ...              loc_parietal loc_occipital   
119            0.0          0.0  ...                       0.0           0.0   
123            0.0          0.0  ...                       0.0           0.0   
124            0.0          1.0  ...                       0.0           0.0   
126            0.0          1.0  ...                       0.0           0.0   
127            0.0          1.0  ...                       0.0           0.0   

                                                               \
    loc_subcortical loc_brainstem loc_cerebellar loc_subdural   
119             0.0           0.0            0.0          0.0   
123             0.0           0.0            0.0          0.0   
124             0.0           0.0            0.0          0.0   
126             0.0           0.0            0.0          0.0   
127             0.0           0.0            0.0          0.0   

                                                 TUGT_average  
    loc_subarachnoid treatment treatment_spec correct_average  
119              0.0       NaN            NaN            10.0  
123              0.0       NaN            NaN             NaN  
124              0.0       NaN            NaN             6.7  
126              0.0       NaN            NaN            10.0  
127              0.0       NaN            NaN             5.3  

[5 rows x 758 columns]

day_365
trail_making_test_b 
 ['TMT_time_taken']
MMSE 
 ['age_assessment' 'pencil' 'watch' 'school_years' 'time_year'
 'time_season' 'time_month' 'time_weekday' 'time_date' 'place_state'
 'place_country' 'place_city' 'place_building' 'place_floor'
 'register_apple' 'register_penny' 'register_table' 'serial7_93'
 'serial7_86' 'serial7_79' 'serial7_72' 'serial7_65' 'world_d' 'world_l'
 'world_r' 'world_o' 'world_w' 'recall_apple' 'recall_penny'
 'recall_table' 'naming_pencil' 'naming_watch' 'repetition'
 'comprehension_take_paper' 'comprehension_fold_paper'
 'comprehension_put_paper' 'reading' 'writing' 'drawing' 'score'
 'consciousness']
SIS 
 ['phy_arm' 'phy_hand' 'phy_leg' 'phy_foot' 'mem_told' 'mem_day_bef'
 'mem_to_do' 'mem_wk_day' 'mem_concert' 'mem_think_quick' 'mem_sol_prob'
 'feel_sad' 'feel_nobody' 'feel_burden' 'feel_nothing' 'feel_blame'
 'feel_enjoy' 'feel_nervous' 'feel_life_with' 'feel_smile' 'com_say_name'
 'com_under' 'com_reply' 'com_correct' 'com_part' 'com_ph' 'com_ph

mRS                   NIHSS                                  \
    score date valid reason   loc loc_quest loc_command best_gaze   
119   2.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
123   3.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
124   1.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
126   2.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   
127   0.0  NaN   1.0    NaN   0.0       0.0         0.0       0.0   

                                 ...      ACS                                 \
    vis_field_test facial_palsy  ... work_pal work_raln_12mth 999 untestable   
119            0.0          0.0  ...      NaN                            NaN   
123            0.0          0.0  ...      0.0                            NaN   
124            0.0          2.0  ...      1.0                          100.0   
126            0.0          1.0  ...      1.0                          100.0   
127            0.0          0.0  ...      1.0                          100.0   

                                                                           \
    continue to do_childcare given up_childcare beginning to do_childcare   
119                      NaN                NaN                       NaN   
123                      0.0                0.0                       0.0   
124                      0.0                0.0                       0.0   
126                      1.0                0.0                       0.0   
127                      1.0                0.0                       0.0   

                                                                         TUGT  
    new_childcare childcare_caln childcare_pal childcare_raln correct_average  
119           NaN            NaN           NaN            NaN             NaN  
123           0.0            0.0           0.0            NaN            30.0  
124           0.0            0.0           0.0            NaN             7.7  
126           0.0            1.0           1.0          100.0             9.3  
127           0.0            1.0           1.0          100.0             6.0  

[5 rows x 763 columns]

In [79]:
flat_index = [c for c in scan_id_df.columns.to_flat_index()]

In [82]:
scan_id_df[flat_index[7:10]]

day_baseline                                           
             NIHSS_multiple recent_medical_history                    
    nihss_other_hr_datetime                    tpa        tpa_datetime
119                     NaT                    NaN                 NaT
123                     NaT                    NaN                 NaT
124                     NaT                    NaN                 NaT
126                     NaT                    NaN                 NaT
127                     NaT                    NaN                 NaT
129         0 days 04:00:00                    1.0 2011-11-14 03:00:00
132                     NaT                    1.0 2012-02-17 03:50:00
133                     NaT                    0.0                 NaT
136       372 days 05:20:00                    0.0                 NaT
137                     NaT                    0.0                 NaT
138         3 days 12:00:00                    0.0                 NaT
139         4 days 11:40:00                    0.0                 NaT
140         4 days 09:10:00                    0.0                 NaT
141                     NaT                    1.0 2012-01-26 15:05:00
142         1 days 14:00:00                    0.0                 NaT
143         1 days 02:30:00                    0.0                 NaT
144                     NaT                    0.0                 NaT
146                     NaT                    0.0                 NaT
147                     NaT                    1.0 2012-04-28 00:50:00
148                     NaT                    0.0                 NaT
151                     NaT                    0.0                 NaT
152                     NaT                    0.0                 NaT
154                     NaT                    1.0 2012-05-15 20:35:00
155         7 days 02:00:00                    0.0                 NaT
156         2 days 17:30:00                    0.0                 NaT
157         0 days 10:20:00                    0.0                 NaT
159      -29 days +15:00:00                    0.0                 NaT
161                     NaT                    0.0                 NaT
162                     NaT                    0.0                 NaT
163                     NaT                    0.0                 NaT
..                      ...                    ...                 ...
178                     NaT                    NaN                 NaT
180                     NaT                    0.0                 NaT
182                     NaT                    0.0                 NaT
183                     NaT                    0.0                 NaT
186         1 days 08:15:00                    0.0                 NaT
188         2 days 00:10:00                    1.0 2013-05-16 07:00:00
189         1 days 22:15:00                    0.0                 NaT
190         1 days 13:40:00                    0.0                 NaT
191                     NaT                    0.0                 NaT
192         0 days 01:10:00                    0.0                 NaT
194                     NaT                    0.0                 NaT
196         0 days 03:45:00                    1.0 2013-07-14 07:30:00
197                     NaT                    1.0 2013-07-20 00:30:00
198                     NaT                    0.0                 NaT
199                     NaT                    0.0                 NaT
200         3 days 01:10:00                    0.0                 NaT
201                     NaT                    0.0                 NaT
202                     NaT                    0.0                 NaT
204                     NaT                    1.0 2013-10-22 23:15:00
205                     NaT                    0.0                 NaT
206                     NaT                    0.0                 NaT
207                     NaT                    0.0                 NaT
208         1 days 00:00:00            

In [86]:
index_key = scan_id_df.columns.to_frame()

In [107]:
index_key[1].unique()

array(['NIHSS_multiple', 'recent_medical_history', 'NIHSS', 'MoCA',
       'MADRS', 'RAPA', 'infarct_type', 'mRS', 'BI', 'SIS', 'WSAS',
       'MMSE', 'digit_span', 'RAVENs', 'STROOP', 'shape_cancellation',
       'TDT', 'ARAT', 'ACS', 'trail_making_test_b',
       'lesion_location_confirmed', 'TUGT_average', 'TUGT'], dtype=object)

In [114]:
index_key.loc['day_baseline']

0  \
NIHSS_multiple         nihss_three_hr_score        day_baseline   
                       nihss_three_hr_datetime     day_baseline   
                       nihss_seven_hr_score        day_baseline   
                       nihss_seven_hr_datetime     day_baseline   
                       nihss_eighteen_hr_score     day_baseline   
                       nihss_eighteen_hr_datetime  day_baseline   
                       nihss_other_hr_score        day_baseline   
                       nihss_other_hr_datetime     day_baseline   
recent_medical_history tpa                         day_baseline   
                       tpa_datetime                day_baseline   
                       tpa_volume                  day_baseline   
NIHSS                  loc                         day_baseline   
                       loc_quest                   day_baseline   
                       loc_command                 day_baseline   
                       best_gaze                   day_baseline   
                       vis_field_test              day_baseline   
                       facial_palsy                day_baseline   
                       motor_arm_l                 day_baseline   
                       motor_arm_r                 day_baseline   
                       motor_leg_l                 day_baseline   
                       motor_leg_r                 day_baseline   
                       limb_ataxia                 day_baseline   
                       sensory                     day_baseline   
                       best_lang                   day_baseline   
                       dysarthria                  day_baseline   
                       extinction                  day_baseline   
                       score                       day_baseline   
                       valid                       day_baseline   
                       reason                      day_baseline   

                                                                        1  \
NIHSS_multiple         nihss_three_hr_score                NIHSS_multiple   
                       nihss_three_hr_datetime             NIHSS_multiple   
                       nihss_seven_hr_score                NIHSS_multiple   
                       nihss_seven_hr_datetime             NIHSS_multiple   
                       nihss_eighteen_hr_score             NIHSS_multiple   
                       nihss_eighteen_hr_datetime          NIHSS_multiple   
                       nihss_other_hr_score                NIHSS_multiple   
                       nihss_other_hr_datetime             NIHSS_multiple   
recent_medical_history tpa                         recent_medical_history   
                       tpa_datetime                recent_medical_history   
                       tpa_volume                  recent_medical_history   
NIHSS                  loc                                          NIHSS   
                       loc_quest                                    NIHSS   
                       loc_command                                  NIHSS   
                       best_gaze                                    NIHSS   
                       vis_field_test                               NIHSS   
                       facial_palsy                                 NIHSS   
                       motor_arm_l                                  NIHSS   
                       motor_arm_r                                  NIHSS   
                       motor_leg_l                                  NIHSS   
                       motor_leg_r                                  NIHSS   
                       limb_ataxia                                  NIHSS   
                       sensory                                      NIHSS   
                       best_lang                                    NIHSS   
                       dysarthria                                   NIHSS   
                       extinction                